# Compare USGS, modeled, and observed ELAs at the BGOTUS

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import glob
import xarray as xr

In [ ]:
# Path to glacier snow cover mapping data
scm_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/'

In [ ]:
# Function to add year and month columns 
def add_year_month_columns(df):
    df['Year'] = pd.DatetimeIndex(df.index).year
    df['Month'] = pd.DatetimeIndex(df.index).month
    return df

## Load remotely-sensed ELAs

In [ ]:
# obs_elas_fn = os.path.join(scm_path, 'compiled_data', 'observed_elas.csv')
# obs_elas = pd.read_csv(obs_elas_fn, index_col=0)
# print('Remotely-sensed ELAs loaded from file.')
# # Add Year and month column
# obs_elas = add_year_month_columns(obs_elas)

# obs_elas
scs_fn = os.path.join(scm_path, 'compiled_data', 'all_snow_cover_stats.csv')
scs = pd.read_csv(scs_fn)
scs['datetime'] = pd.to_datetime(scs['datetime'])
scs['Year'] = pd.DatetimeIndex(scs['datetime']).year
scs['Month'] = pd.DatetimeIndex(scs['datetime']).month
scs

## Load BGOTUS ELAs

In [ ]:
bgotus_path = os.path.join(scm_path, 'benchmarkGlacier_massBalance')

name_rgi_id_dict = {'Gulkana': 'RGI60-01.00570',
                    'Wolverine': 'RGI60-01.09162',
                    'LemonCreek': 'RGI60-01.01104',
                    'Taku': 'RGI60-01.01390',
                    'SouthCascade': 'RGI60-02.18778',
                    'Sperry': 'RGI60-02.17023'}

site_names = list(name_rgi_id_dict.keys())
elas_usgs = pd.DataFrame()
for site_name in site_names:
    elas_fn = os.path.join(bgotus_path, site_name, f'Output_{site_name}_Glacier_Wide_solutions_calibrated.csv')
    elas = pd.read_csv(elas_fn)
    elas['Name'] = site_name
    elas['RGIId'] = name_rgi_id_dict[site_name]
    elas_usgs = pd.concat([elas_usgs, elas])
elas_usgs = elas_usgs.dropna().reset_index(drop=True)

# Add year and month columns
elas_usgs['Ba_Date'] = pd.to_datetime(elas_usgs['Ba_Date'], format='mixed')
elas_usgs.set_index('Ba_Date', inplace=True)
elas_usgs = add_year_month_columns(elas_usgs)

elas_usgs

## Load modeled ELAs

In [ ]:
mod_elas_fn = os.path.join(scm_path, 'Rounce_et_al_2023', 'modeled_elas.csv')
mod_elas = pd.read_csv(mod_elas_fn, index_col=0)
print('Modeled ELAs loaded from file.')
# Add year and month columns
mod_elas = add_year_month_columns(mod_elas)

mod_elas

## Compare annual ELAs

In [ ]:
# Iterate over BGOTUS RGI IDs
plt.rcParams.update({'font.size':12, 'font.sans-serif':'Arial'})
for name, rgi_id in elas_usgs[['Name', 'RGIId']].drop_duplicates().values:
    print(rgi_id, name)
    
    # subset data to site
    elas_usgs_site = elas_usgs.loc[elas_usgs['RGIId']==rgi_id]
    mod_elas_site = mod_elas[[rgi_id, 'Year', 'Month']].reset_index()
    obs_elas_site = scs.loc[scs['RGIId']==rgi_id]
    obs_elas_site.loc[obs_elas_site['ELA_from_AAR_m'] > 1e30, 'ELA_from_AAR_m'] = np.nan

    # subtract the minimum elevation to remove any datum biases
    # elas_usgs_site['ELA'] -= np.nanmin(obs_elas[rgi_id])
    # mod_elas_site[rgi_id] -= np.nanmin(mod_elas[rgi_id])
    # obs_elas_site[rgi_id] -= np.nanmin(obs_elas[rgi_id])
    
    # plot
    fig, ax = plt.subplots(1, 2, figsize=(12,6), sharey=True)
    # Transient ELAs
    ax[0].plot(pd.DatetimeIndex(mod_elas.index), mod_elas_site[rgi_id], '^-c', label='Modeled')
    ax[0].plot(obs_elas_site['datetime'], obs_elas_site['ELA_from_AAR_m'], '.b', label='Remotely-sensed')
    ax[0].plot(pd.DatetimeIndex(elas_usgs_site.index), elas_usgs_site['ELA'], '*-g', label='USGS')
    ax[0].set_xlim(np.datetime64('2010-01-01'), np.datetime64('2024-01-01'))
    ax[0].set_ylabel('meters')
    ax[0].set_title('Transient ELAs')
    ax[0].grid()
    # Annual ELAs
    ax[1].plot(mod_elas_site.groupby('Year')[rgi_id].max().index,
               mod_elas_site.groupby('Year')[rgi_id].max().values,
               '^-c', label='Modeled')
    ax[1].plot(obs_elas_site.groupby('Year')['ELA_from_AAR_m'].max().index,
               obs_elas_site.groupby('Year')['ELA_from_AAR_m'].max().values,
               '.-b', label='Remotely-sensed')   
    ax[1].plot(elas_usgs_site.groupby('Year')['ELA'].max().index,
               elas_usgs_site.groupby('Year')['ELA'].max().values,
               '*-g', label='USGS')     
    ax[1].set_title('ELAs')
    ax[1].grid()
    ax[1].set_xlim(2010, 2024)

    handles, labels = ax[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', markerscale=2, ncols=3)

    fig.suptitle(f'{rgi_id}: {name}')
    plt.show()

In [ ]:
import rioxarray as rxr
dem_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites/RGI60-01.09162/DEMs/Wolverine_ArcticDEM_clip_geoid.tif'
dem = rxr.open_rasterio(dem_fn)
dem

In [ ]:
np.nanmin(dem.data[0]), np.nanmax(dem.data[0])